### Describtion

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: unreliable

0: reliable

In [2]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [3]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Stop words are words which occur frequently in a corpus. e.g a, an, the, in. Frequently 
occurring words are removed from the corpus for the sake of text-normalization.


### Data preprocessing

In [4]:
train = pd.read_csv('train.csv')
train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


### Handling missing data

In [5]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
train.fillna('', inplace=True)

In [8]:
train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

### merging author name and news title
as we will use these two columns in the prediction

In [9]:
train['content'] = train['author']+' '+train['title']
train

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


### Stemming

stemming is the process of reducing inflected words to their word stem, base or root form

In [10]:
#ps = PorterStemmer()
#lc = LancasterStemmer()
snow = SnowballStemmer(language='english')

In [11]:
def stemming(text):
    stemmed_text = re.sub('[^a-zA-Z]', ' ', text )  #removes everything except Alphabets
    stemmed_text = stemmed_text.lower()             #convert the words into lowercase
    stemmed_text = stemmed_text.split()             #Split the words to list
    stemmed_text = [snow.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
    stemmed_text = ' '.join(stemmed_text)
    return stemmed_text

In [12]:
train['content'] = train['content'].apply(stemming)

In [13]:
train['content']

0        darrel lucus hous dem aid even see comey lette...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [14]:
x = train.drop(columns='label', axis=1)
y = train['label']

In [16]:
x = train['content'].values
x

array(['darrel lucus hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campus breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [17]:
y = train['label'].values
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

### Converting the Textual Data to Numerical Data

tf-idf stands for Term Frequency – Inverse Document Frequency 

tf is the number of times a term appears in a particular document. So it’s specific to a document

idf is a measure of how common or rare a term is across the entire corpus of documents. So the point to note is that it’s common to all the documents. If the word is common and appears in many documents, the idf value (normalized) will approach 0 or else approach 1 if it’s rare

In [18]:
vectorizer = TfidfVectorizer()

In [19]:
vectorizer.fit(x)
x = vectorizer.transform(x)

In [21]:
print(x)

  (0, 15626)	0.28486264020973145
  (0, 13421)	0.25660048150216447
  (0, 8881)	0.3636117038307405
  (0, 8603)	0.29213745202547464
  (0, 7664)	0.24786264054802615
  (0, 6976)	0.21875090941843942
  (0, 4953)	0.23314610616186185
  (0, 3774)	0.2705446492831935
  (0, 3583)	0.35990908598995375
  (0, 2943)	0.24685541574856376
  (0, 2470)	0.3676674627942394
  (0, 265)	0.2701126327698421
  (1, 16734)	0.30245489916555496
  (1, 6788)	0.19156646734771887
  (1, 5475)	0.7184570896190878
  (1, 3550)	0.2663102550869915
  (1, 2797)	0.19204895980429573
  (1, 2212)	0.36908226044961334
  (1, 1885)	0.15611654884493256
  (1, 1490)	0.2953721148085586
  (2, 15550)	0.41544962664721613
  (2, 9591)	0.49351492943649944
  (2, 5944)	0.3474613386728292
  (2, 5362)	0.3866530551182615
  (2, 3087)	0.46097489583229645
  :	:
  (20797, 13073)	0.24792105912992887
  (20797, 12298)	0.27227043505318577
  (20797, 12094)	0.24745162898225476
  (20797, 10276)	0.0802734302266876
  (20797, 9558)	0.17432033968695243
  (20797, 9489)	0

### Spliting the data into training and testing data

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

### Modeling

In [27]:
def train(model, model_name):
    model.fit(x_train, y_train)
    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)
    print(f"{model_name} score on train: {train_score * 100}%\n{model_name} score on test: {test_score}%")

### Logistic Regression

In [28]:
lr = LogisticRegression()
train(lr, 'Logistic Regression')

Logistic Regression score on train: 98.65985576923076%
Logistic Regression score on test: 0.9730769230769231%


### Random Forest

In [30]:
rf = RandomForestClassifier(n_estimators=80, random_state=10)
train(rf, "Random Forest")

Random Forest score on train: 100.0%
Random Forest score on test: 0.9930288461538461%


### KNN

In [34]:
knn = KNeighborsClassifier(n_neighbors=3)
train(knn, "KNN")

KNN score on train: 58.88822115384615%
KNN score on test: 0.5420673076923077%


### Naive Bayes

In [35]:
naive_bayes = MultinomialNB()
train(naive_bayes, "Naive Bayes")

Naive Bayes score on train: 96.97115384615384%
Naive Bayes score on test: 0.9471153846153846%
